In [7]:
ls

$RECYCLE.BIN/
113391.pdf
2.【入力例】学位授与申請書.pdf
2021年度秋学期TA登録に関わる留意事項.pdf
Logi Capture Installer 2.08.12.app/
Toggl_Track_summary_report_2022-02-07_2022-02-13.pdf
bbcd75aa-dfbe-4154-acca-82375ca3c41b
desktop.ini
kanban_animation.mp4
progex1_02 (1).pdf
progex1_02.pdf
progex1_08.pdf
sample1.csv
sample2.csv
sample3.csv
データ作成_設計値最新データ.ipynb


In [25]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
#ライブラリのimport
import pandas as pd
import numpy as np
%matplotlib inline
#from matplotlib import pyplot as pyp
import matplotlib.pyplot as plt

In [2]:
#データ準備
#手順
#
pre_zaikodata = pd.read_excel('..//data//設計値変更前_所在管理LT.xlsx',encoding='shift_jis')
#ひとまずダミーデータで。
#pre_zaikodata = pd.read_csv('..//data//pre_zaikodata_20230828.csv',encoding='shift_jis')
#pre_zaikodata = pd.read_csv('sample1.csv',encoding='shift_jis')
pre_zaikodata.head(300)

TypeError: read_excel() got an unexpected keyword argument 'encoding'

In [28]:
#確認用（実行は不要）
pre_zaikodata.isnull().sum()

かんばんシリアル          3
伝票番号              3
品番                3
品名                3
収容数               3
仕入先名              3
仕入先工場名            3
ステータス             3
納入日               3
納入便               3
発注日時              3
印刷日時              3
検収日時              3
順立装置入庫日時          3
順立装置出庫日時          3
組立投入日時            3
回収日時              3
発注取消日時            3
発注印刷LT            3
発注検収LT           85
発注順立装置入庫LT        4
発注順立装置出庫LT       12
発注組立LT        12542
発注回収LT            3
更新日時              3
長期滞留フラグ           1
dtype: int64

In [29]:
###LT

long = pre_zaikodata.iloc[:,1]
count = 0
for i in range(len(long)):
    if i % 5000==0:
            print(i)
    if ((pre_zaikodata.loc[i, '発注印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注回収LT'])!='< NULL >'):
    #if (not pd.isnull(pre_zaikodata.loc[i, '発注印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注回収LT'])):
        LT2 = float(pre_zaikodata.loc[i, '発注検収LT']) - float(pre_zaikodata.loc[i, '発注印刷LT'])
        LT3 = float(pre_zaikodata.loc[i, '発注順立装置入庫LT'])-float(pre_zaikodata.loc[i, '発注検収LT'])
        LT4 = float(pre_zaikodata.loc[i, '発注順立装置出庫LT'])-float(pre_zaikodata.loc[i, '発注順立装置入庫LT'])
        LT5 = float(pre_zaikodata.loc[i, '発注回収LT'])-float(pre_zaikodata.loc[i, '発注順立装置出庫LT'])
        LT6 = float(pre_zaikodata.loc[i, '発注回収LT'])-float(pre_zaikodata.loc[i, '発注検収LT'])
        pre_zaikodata.loc[i, '印刷検収LT'] = LT2
        pre_zaikodata.loc[i, '検収入庫LT'] = LT3
        pre_zaikodata.loc[i, '入庫出庫LT'] = LT4
        pre_zaikodata.loc[i, '出庫回収LT'] = LT5
        pre_zaikodata.loc[i, '社内LT'] = LT6
        count = count + 1
print(count)

0
5000
10000
12542


In [30]:
###基準在庫枚数加味する

#稼働日入力
#kadoubi=22

count = 0
tehaidata = pd.read_csv('..//data//設計値変更前_202303_手配数_1Y_8次_data.csv',encoding='cp932')
#tehaidata = pd.read_csv('sample3.csv',encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    if ((pre_zaikodata.loc[i, '発注印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注回収LT'])!='< NULL >'):
    #if (not pd.isnull(pre_zaikodata.loc[i, '発注印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注回収LT'])):
        hinban = str(pre_zaikodata.loc[i,'品番'] )
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        #print(i)
        if i % 5000==0:
                print(i)
        for j in range(len(long2)):
            #print(j)
            tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
            tehaihinban2 = tehaihinban.replace(' ', '')
            if hinban3 == tehaihinban2:
                if int(str(tehaidata.loc[j,'当月計算数']).replace(',', '')) != 0:
                    avenichiryo = int(str(tehaidata.loc[j,'当月計算数']).replace(',', ''))/int(tehaidata.loc[j,'当月稼働日数'])
                    pre_zaikodata.loc[i,'平均日量数'] = avenichiryo
                    pre_zaikodata.loc[i,'基準在庫枚数日数'] = (int(tehaidata.loc[j,'基準在庫枚数'])*pre_zaikodata.loc[i, '収容数'])/avenichiryo
                else:
                    pre_zaikodata.loc[i,'平均日量数'] = 0
                    pre_zaikodata.loc[i,'基準在庫枚数日数'] = 0
print(count)

0
5000
10000
0


In [31]:
count = 0
tehaidata = pd.read_excel('..//data//設計値変更前_XR10_手配運用情報_データ抽出.xlsx',encoding='cp932')
#tehaidata = pd.read_csv('sample2.csv',encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    if (not pd.isnull(pre_zaikodata.loc[i, '発注印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注回収LT'])):
        hinban = str(pre_zaikodata.loc[i,'品番'] )
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        if i % 5000==0:
            print(i,hinban2)
        for j in range(len(long2)):
            tehaihinban = str(tehaidata.loc[j,'品番']).replace('-', '')
            tehaihinban2 = tehaihinban.replace(' ', '')
            #print(tehaihinban)
            if hinban3 == tehaihinban2:
                pre_zaikodata.loc[i, 'A'] = tehaidata.loc[j,'納入ｻｲｸﾙ間隔']
                pre_zaikodata.loc[i, 'B'] = tehaidata.loc[j,'納入ｻｲｸﾙ回数']
                pre_zaikodata.loc[i, 'C'] = tehaidata.loc[j,'納入ｻｲｸﾙ情報']
                pre_zaikodata.loc[i, '整備室'] = tehaidata.loc[j,'整備室']
                pre_zaikodata.loc[i, '箱種'] = tehaidata.loc[j,'登録箱種']
                pre_zaikodata.loc[i, '箱サイズ'] = tehaidata.loc[j,'容積(m3)']
                pre_zaikodata.loc[i, '箱重量'] = tehaidata.loc[j,'総重量(Kg)']
                #pre_zaikodata.loc[i, '不当ピッチ'] = tehaidata.loc[j,'不当ピッチ']
                #pre_zaikodata.loc[i, '通常品'] = tehaidata.loc[j,'通常品']
                #pre_zaikodata.loc[i, '選択品'] = tehaidata.loc[j,'選択品']
                pre_zaikodata.loc[i, '基準在庫日数'] = tehaidata.loc[j,'基準在庫日数']
                pre_zaikodata.loc[i, '基準在庫枚数'] = tehaidata.loc[j,'基準在庫枚数']
                pre_zaikodata.loc[i, '手配区分'] = tehaidata.loc[j,'手配区分']
                MAX= tehaidata.loc[j,'基準在庫日数'] + float(pre_zaikodata.loc[i,'基準在庫枚数日数'])
                pre_zaikodata.loc[i,'基準在庫日数（基準在庫枚数込み）'] = MAX
                zisseki = pre_zaikodata.loc[i, '社内LT'] 
                pre_zaikodata.loc[i, '実績在庫日数'] = zisseki
                if  (zisseki > MAX*0)and(zisseki < 0.25*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常1'
                elif  (zisseki > MAX*0.25)and(zisseki < 0.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常2'
                elif  (zisseki > MAX*0.5)and(zisseki < 0.75*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常3'
                elif  (zisseki > MAX*0.75)and(zisseki < MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常4'
                elif (zisseki >= MAX)and(zisseki < MAX*1.25):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常1'
                elif (zisseki >= MAX*1.25)and(zisseki < 1.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常2'
                elif (zisseki >= MAX*1.5)and(zisseki < 2*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常3'
                elif (zisseki >= MAX*2)and(zisseki < 3*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常4'
                elif zisseki >= 3*MAX:
                    pre_zaikodata.loc[i, '在庫状態'] = '異常5'
        #print(LT2,LT3,LT4,LT5)
        count = count + 1
print(count)

5000 35557ECB010
10000 33490ECE010
12447


In [32]:
with open('..//data//設計値変更前_LTデータ.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    pre_zaikodata.to_csv(f)

In [19]:
#pre_zaikodata.head(300)
tehaidata.head(300)

,タイトル：かんばん手配品番一覧,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ダウンロード日時：2023/03/1515:15:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,手配担当工場：＜第１＞；手配担当整備室：＜第１工場＞；発注区分：＜かんばん＞；手配区分：＜仕...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,9056451A090,NaN,NaN,SHIM,1Y,第１工場,手配,Ｐ,6454,（株）ムロコーポレーション,...,0.006,4.680,しない,NaN,MASTER01,2022/04/19,NE00,１工工務Ｇ,46548,長畑 玲奈
296,9056451A091,NaN,NaN,SHIM,1Y,第１工場,手配,Ｐ,6454,（株）ムロコーポレーション,...,0.006,6.500,しない,NaN,MASTER01,2022/04/19,NE00,１工工務Ｇ,46548,長畑 玲奈
297,9056451A092,NaN,NaN,SHIM,1Y,第１工場,手配,Ｐ,6454,（株）ムロコーポレーション,...,0.006,6.500,しない,NaN,MASTER01,2022/04/19,NE00,１工工務Ｇ,46548,長畑 玲奈
298,9056451A093,NaN,NaN,SHIM,1Y,第１工場,手配,Ｐ,6454,（株）ムロコーポレーション,...,0.006,6.500,しない,NaN,MASTER01,2022/04/19,NE00,１工工務Ｇ,46548,長畑 玲奈


In [9]:
7/3

2.3333333333333335